# Lets Scrape All the ESPN Win Percentages

In [3]:
# Lets do this!

import requests
from bs4 import BeautifulSoup

## First Pull All the IDS

In [13]:
400951840 - 400951552

288

In [14]:
first_gameid = 400951552
#next_gameid = 400951601

gameids = []

for x in range(290):
    if x == 0:
        gameid = first_gameid
    else:
        gameid += 1
    page = requests.get('http://www.espn.com/nfl/game?gameId={}'.format (gameid))
    soup = BeautifulSoup(page.text, 'html.parser')
    results = soup.findAll(name='title')
    print(gameid,results[0])
    gameids.append(gameid)

400951552 <title>Patriots vs. Buccaneers - Game Summary - October 5, 2017 - ESPN</title>
400951553 <title>Seahawks vs. Cardinals - Game Summary - November 9, 2017 - ESPN</title>
400951554 <title>Bills vs. Bengals - Game Summary - October 8, 2017 - ESPN</title>
400951555 <title>Saints vs. Bills - Game Summary - November 12, 2017 - ESPN</title>
400951556 <title>Jets vs. Browns - Game Summary - October 8, 2017 - ESPN</title>
400951557 <title>Saints vs. Falcons - Game Summary - December 7, 2017 - ESPN</title>
400951558 <title>Panthers vs. Lions - Game Summary - October 8, 2017 - ESPN</title>
400951559 <title>Packers vs. Bears - Game Summary - November 12, 2017 - ESPN</title>
400951560 <title>Colts vs. Bills - Game Summary - December 10, 2017 - ESPN</title>
400951561 <title>Vikings vs. Panthers - Game Summary - December 10, 2017 - ESPN</title>
400951562 <title>49ers vs. Colts - Game Summary - October 8, 2017 - ESPN</title>
400951563 <title>Browns vs. Lions - Game Summary - November 12, 2017

400951648 <title>Chargers vs. Jaguars - Game Summary - November 12, 2017 - ESPN</title>
400951649 <title>49ers vs. Texans - Game Summary - December 10, 2017 - ESPN</title>
400951650 <title>Chargers vs. Giants - Game Summary - October 8, 2017 - ESPN</title>
400951651 <title>Cardinals vs. Eagles - Game Summary - October 8, 2017 - ESPN</title>
400951652 <title>Seahawks vs. Jaguars - Game Summary - December 10, 2017 - ESPN</title>
400951653 <title>Jets vs. Buccaneers - Game Summary - November 12, 2017 - ESPN</title>
400951654 <title>Raiders vs. Chiefs - Game Summary - December 10, 2017 - ESPN</title>
400951655 <title>Jaguars vs. Steelers - Game Summary - October 8, 2017 - ESPN</title>
400951656 <title>Bengals vs. Titans - Game Summary - November 12, 2017 - ESPN</title>
400951657 <title>Seahawks vs. Rams - Game Summary - October 8, 2017 - ESPN</title>
400951658 <title>Vikings vs. Redskins - Game Summary - November 12, 2017 - ESPN</title>
400951659 <title>Ravens vs. Raiders - Game Summary - 

400951747 <title>Colts vs. Seahawks - Game Summary - October 1, 2017 - ESPN</title>
400951748 <title>Bills vs. Dolphins - Game Summary - December 31, 2017 - ESPN</title>
400951749 <title>Falcons vs. Panthers - Game Summary - November 5, 2017 - ESPN</title>
400951750 <title>Bears vs. Vikings - Game Summary - December 31, 2017 - ESPN</title>
400951751 <title>Colts vs. Texans - Game Summary - November 5, 2017 - ESPN</title>
400951752 <title>Redskins vs. Chiefs - Game Summary - October 2, 2017 - ESPN</title>
400951753 <title>Bengals vs. Jaguars - Game Summary - November 5, 2017 - ESPN</title>
400951754 <title>Steelers vs. Bengals - Game Summary - December 4, 2017 - ESPN</title>
400951755 <title>Buccaneers vs. Saints - Game Summary - November 5, 2017 - ESPN</title>
400951756 <title>Jets vs. Patriots - Game Summary - December 31, 2017 - ESPN</title>
400951757 <title>Redskins vs. Giants - Game Summary - December 31, 2017 - ESPN</title>
400951758 <title>Rams vs. Giants - Game Summary - Novembe

In [16]:
len(gameids)

290

## Scripts to Pull Win Prob Data

In [60]:
from lxml import html
from lxml.cssselect import CSSSelector
import requests

import pandas as pd

import json

css = CSSSelector

win_pct_marker = "espn.gamepackage.probability.data = "

def is_wins(x):
    return win_pct_marker in x

def extract_other_wins(doc):
    scripts = css("script")(doc)
    target_script = next(script for script in scripts if is_wins(script.text_content()))
    js = target_script.text_content()
    target_line = next(line for line in js.splitlines() if is_wins(line))
    start = target_line.find(win_pct_marker)
    data = target_line[start + len(win_pct_marker):-1]
    parsed = json.loads(data)
    return parsed

def extract_game_data(src):
    doc = html.fromstring(src)
    primary_win_pct = css(".header-win-percentage")(doc)[0].text_content()
    game_title = css("title")(doc)[0].text_content()
    other_win_pcts = extract_other_wins(doc)
    return (primary_win_pct, other_win_pcts), game_title

def convert_gamedata_to_df(score):
    df = pd.DataFrame(score[1])
    df['awayScore'], df['clockdisplayValue'], df['homeScore'], \
    df['periodnumber'], df['startdistance'], df['startdown'], df['startteamid'], \
    df['startyardLine'], df['startyardsToEndzone'], df['playtext'], \
    df['playtypeabbreviation'], df['playtypeid'] \
    = zip(*df['play'].map(explode_play_col))
    #, df['playtypetext'] 
    
    df = df.drop('play', axis=1)
    return df

def explode_play_col(col):
    awayScore = col['awayScore']
    clockdisplayValue = col['clock']['displayValue']
    homeScore = col['homeScore']
    periodnumber = col['period']['number']
    startdistance = col['start']['distance']
    startdown = col['start']['down']
    try:
        startteamid = col['start']['team']['id']
    except:
        startteamid = None
    startyardLine = col['start']['yardLine']
    startyardsToEndzone = col['start']['yardsToEndzone']
    
    try:
        playtext = col['text']
    except:
        playtext = None
    
    try:
        playtypeabbreviation = col['type']['abbreviation']
    except:
        playtypeabbreviation = None
        
    try:
        playtypeid = col['type']['id']
    except:
        playtypeid = None
    
    try:
        playtypetext = col['type']['text']
    except:
        playtypetext = ' '
    return awayScore, clockdisplayValue, homeScore, periodnumber, startdistance, startdown, \
            startteamid, startyardLine, startyardsToEndzone, playtext, playtypeabbreviation, \
            playtypeid#, playtypetextxt

## Loop Through Every Game

In [79]:
for gameid in gameids:
    print("Running for gameid {}".format(gameid))
    try:
        url = 'http://www.espn.com/nfl/game?gameId={}'.format(gameid)


        src = requests.get(url)
        score, game_title = extract_game_data(src.text)

        df = convert_gamedata_to_df(score)
        df['Game Title'] = game_title.split('-')[0]
        df['Game Date'] = game_title.split('-')[2]
        df.to_csv('data/{}-{}-WinProb.csv'.format(game_title.split('-')[0].replace(' ',''),
                                                 game_title.split('-')[2].replace(' ','')))

        print("Finished for {} {}".format(game_title.split('-')[0].replace(' ',''),
                                                 game_title.split('-')[2].replace(' ','')))
    except:
        print("FAILED for {} {}".format(game_title.split('-')[0].replace(' ',''),
                                                 game_title.split('-')[2].replace(' ','')))

Running for gameid 400951552
Finished for Patriotsvs.Buccaneers October5,2017
Running for gameid 400951553
Finished for Seahawksvs.Cardinals November9,2017
Running for gameid 400951554
Finished for Billsvs.Bengals October8,2017
Running for gameid 400951555
Finished for Saintsvs.Bills November12,2017
Running for gameid 400951556
Finished for Jetsvs.Browns October8,2017
Running for gameid 400951557
Finished for Saintsvs.Falcons December7,2017
Running for gameid 400951558
Finished for Panthersvs.Lions October8,2017
Running for gameid 400951559
Finished for Packersvs.Bears November12,2017
Running for gameid 400951560
Finished for Coltsvs.Bills December10,2017
Running for gameid 400951561
Finished for Vikingsvs.Panthers December10,2017
Running for gameid 400951562
Finished for 49ersvs.Colts October8,2017
Running for gameid 400951563
Finished for Brownsvs.Lions November12,2017
Running for gameid 400951564
Finished for Bearsvs.Bengals December10,2017
Running for gameid 400951565
Finished for 

Finished for Texansvs.Rams November12,2017
Running for gameid 400951664
Finished for Chiefsvs.Texans October8,2017
Running for gameid 400951665
Finished for Jetsvs.Broncos December10,2017
Running for gameid 400951666
Finished for Dolphinsvs.Chargers September17,2017
Running for gameid 400951667
Finished for Ramsvs.Titans December24,2017
Running for gameid 400951668
Finished for Cowboysvs.Cardinals September25,2017
Running for gameid 400951669
Finished for Jetsvs.Raiders September17,2017
Running for gameid 400951670
FAILED for Jetsvs.Raiders September17,2017
Running for gameid 400951671
Finished for Broncosvs.Redskins December24,2017
Running for gameid 400951672
Finished for Redskinsvs.Cowboys November30,2017
Running for gameid 400951673
Finished for Cowboysvs.Broncos September17,2017
Running for gameid 400951674
Finished for Redskinsvs.Rams September17,2017
Running for gameid 400951675
Finished for Jaguarsvs.49ers December24,2017
Running for gameid 400951676
Finished for 49ersvs.Seahaw

Finished for Ramsvs.Vikings November19,2017
Running for gameid 400951776
FAILED for Ramsvs.Vikings November19,2017
Running for gameid 400951777
Finished for Packersvs.Panthers December17,2017
Running for gameid 400951778
Finished for Ravensvs.Browns December17,2017
Running for gameid 400951779
Finished for Chargersvs.Raiders October15,2017
Running for gameid 400951780
Finished for Redskinsvs.Saints November19,2017
Running for gameid 400951781
Finished for Texansvs.Jaguars December17,2017
Running for gameid 400951782
Finished for Giantsvs.Broncos October15,2017
Running for gameid 400951783
Finished for Chiefsvs.Giants November19,2017
Running for gameid 400951784
Finished for Bengalsvs.Vikings December17,2017
Running for gameid 400951785
Finished for Coltsvs.Titans October16,2017
Running for gameid 400951786
Finished for Chiefsvs.Cowboys November5,2017
Running for gameid 400951787
Finished for Raidersvs.Dolphins November5,2017
Running for gameid 400951788
Finished for Jaguarsvs.Titans De

## Playoff Games

In [81]:
gameid = 400999172 # Patriots Jags

In [82]:
url = 'http://www.espn.com/nfl/game?gameId={}'.format(gameid)


src = requests.get(url)
score, game_title = extract_game_data(src.text)

df = convert_gamedata_to_df(score)
df['Game Title'] = game_title.split('-')[0]
df['Game Date'] = game_title.split('-')[2]
df.to_csv('data/{}-{}-WinProb.csv'.format(game_title.split('-')[0].replace(' ',''),
                                         game_title.split('-')[2].replace(' ','')))

print("Finished for {} {}".format(game_title.split('-')[0].replace(' ',''),
                                         game_title.split('-')[2].replace(' ','')))

Finished for Jaguarsvs.Patriots January21,2018
